In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [3]:
import awq

In [4]:
import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from functools import partial
import gc

In [5]:
def evaluate(model, tokenizer):
    testenc = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
    testenc = tokenizer("\n\n".join(testenc["text"]), return_tensors="pt")

    testenc = testenc.input_ids.to(model.device)
    nsamples = 40
    model = model.eval()

    nlls = []
    for i in tqdm.tqdm(range(nsamples), desc="evaluating..."):
        batch = testenc[:, (i * 2048) : ((i + 1) * 2048)].to(model.device)
        with torch.no_grad():
            lm_logits = model(batch).logits
        shift_logits = lm_logits[:, :-1, :].contiguous().float()
        shift_labels = testenc[:, (i * 2048) : ((i + 1) * 2048)][:, 1:]
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(
            shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1)
        )
        neg_log_likelihood = loss.float() * 2048
        nlls.append(neg_log_likelihood)

    return torch.exp(torch.stack(nlls).sum() / (nsamples * 2048))

In [6]:
def get_model_size(model: nn.Module, data_width=16, group_size=-1):

    if group_size != -1:
        data_width += (16 + 4) / group_size

    num_elements = 0
    for param in model.parameters():
        num_elements += param.numel()
    return num_elements * data_width


Byte = 8
KiB = 1024 * Byte
MiB = 1024 * KiB
GiB = 1024 * MiB

In [7]:
model_path = "facebook/opt-1.3b"

# model_path = "facebook/opt-13b"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

/home/sizheli/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
original_model_n_bits = 32
torch_dtype = torch.float16 if original_model_n_bits == 16 else torch.float32

model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch_dtype, device_map="auto"
)

/home/sizheli/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# Evaluate the model
model_perplexity = evaluate(model, tokenizer)
model_size = get_model_size(model, data_width=original_model_n_bits, group_size=128)

### Print the results
print(f"\nmodel perplexity: {model_perplexity:.2f}")
print(f"model size: {model_size/MiB:.2f} MiB")

evaluating...: 100%|██████████| 40/40 [00:20<00:00,  1.93it/s]



model perplexity: 14.47
model size: 5043.73 MiB


In [48]:
from awq.quantize.pre_quant_lester import run_awq, apply_awq
from typing import Literal

ActQuantType = Literal["per_token", "per_tensor", "none"]

q_config = {
    "zero_point": True,  # by default True
    "q_group_size": 256,  # whether to use group quantization
    "w_n_bits": 4,
    "a_n_bits": 4,
    "act_quant": "per_tensor",
}

model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch_dtype, device_map="auto"
)

awq_results = run_awq(
    model,
    tokenizer,
    w_bit=q_config["w_n_bits"],
    q_config=q_config,
    n_samples=128,
    seqlen=512,
)

Repo card metadata block was not found. Setting CardData to empty.


 * Split into 60 blocks


Running AWQ...:   0%|          | 0/24 [00:00<?, ?it/s]

dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
----------
ratio: 0.0, loss: 1.0621346235275269
ratio: 0.05, loss: 1.0885839462280273
ratio: 0.1, loss: 0.985962986946106
ratio: 0.15, loss: 0.9112263917922974
ratio: 0.2, loss: 0.9200567603111267
ratio: 0.25, loss: 0.9309477806091309
ratio: 0.3, loss: 0.9278945326805115
ratio: 0.35, loss: 0.9316182732582092
ratio: 0.4, loss: 0.9297643303871155
ratio: 0.45, loss: 0.9304285049438477
ratio: 0.5, loss: 0.9303783774375916
ratio: 0.55, loss: 0.9310886859893799
ratio: 0.6, loss: 0.9300938248634338
ratio: 0.65, loss: 0.930308997631073
ratio: 0.7, loss: 0.9299536347389221
ratio: 0.75, loss: 0.9301728010177612
ratio: 0.8, loss: 0.9302728772163391
ratio: 0.85, loss: 0.9302891492843628
ratio: 0.9, loss: 0.9302465319633484
ratio: 0.95, loss: 0.9301099181175232
ratio: 0.0, loss: 0.01421

Running AWQ...:   4%|▍         | 1/24 [00:08<03:18,  8.62s/it]

...

Running AWQ...:  92%|█████████▏| 22/24 [02:44<00:14,  7.43s/it]

dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
----------
ratio: 0.0, loss: 4.91106653213501
ratio: 0.05, loss: 4.911135673522949
ratio: 0.1, loss: 4.911931037902832
ratio: 0.15, loss: 4.911045551300049
ratio: 0.2, loss: 4.911061763763428
ratio: 0.25, loss: 4.911036014556885
ratio: 0.3, loss: 4.909476280212402
ratio: 0.35, loss: 4.90964937210083
ratio: 0.4, loss: 4.909162521362305
ratio: 0.45, loss: 4.9094038009643555
ratio: 0.5, loss: 4.90944766998291
ratio: 0.55, loss: 4.909997940063477
ratio: 0.6, loss: 4.9101948738098145
ratio: 0.65, loss: 4.909687042236328
ratio: 0.7, loss: 4.909558296203613
ratio: 0.75, loss: 4.90934944152832
ratio: 0.8, loss: 4.908905982971191
ratio: 0.85, loss: 4.90927267074585
ratio: 0.9, loss: 4.90928316116333
ratio: 0.95, loss: 4.909463882446289
ratio: 0.0, loss: 0.2129235714673996
ratio: 0.0

Running AWQ...:  96%|█████████▌| 23/24 [02:53<00:07,  7.81s/it]

dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
dict_keys(['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.out_proj', 'fc1', 'fc2'])
----------
ratio: 0.0, loss: 4.252490043640137
ratio: 0.05, loss: 4.252146244049072
ratio: 0.1, loss: 4.2516350746154785
ratio: 0.15, loss: 4.251548767089844
ratio: 0.2, loss: 4.252331256866455
ratio: 0.25, loss: 4.2508225440979
ratio: 0.3, loss: 4.251089096069336
ratio: 0.35, loss: 4.2515482902526855
ratio: 0.4, loss: 4.2508134841918945
ratio: 0.45, loss: 4.251100063323975
ratio: 0.5, loss: 4.250777721405029
ratio: 0.55, loss: 4.25013542175293
ratio: 0.6, loss: 4.250169277191162
ratio: 0.65, loss: 4.250665187835693
ratio: 0.7, loss: 4.250792503356934
ratio: 0.75, loss: 4.25083065032959
ratio: 0.8, loss: 4.250692367553711
ratio: 0.85, loss: 4.2504777908325195
ratio: 0.9, loss: 4.250931739807129
ratio: 0.95, loss: 4.251259803771973
ratio: 0.0, loss: 0.17429298162460327
ratio

Running AWQ...: 100%|██████████| 24/24 [03:00<00:00,  7.52s/it]


In [49]:
dump_awq = "awq_results.pt"
torch.save(awq_results, dump_awq)
print("AWQ results saved at", dump_awq)
#

AWQ results saved at awq_results.pt


In [50]:
### load awq

load_awq = "awq_results.pt"
awq_results = torch.load(load_awq, map_location="cpu")

In [51]:
q_config

{'zero_point': True,
 'q_group_size': 256,
 'w_n_bits': 4,
 'a_n_bits': 4,
 'act_quant': 'per_tensor'}

In [52]:

from transformers.models.opt.modeling_opt import OPTDecoderLayer
from awq.quantize.fake_quant_lester import quantize_opt_model

model = AutoModelForCausalLM.from_pretrained(
    model_path, torch_dtype=torch_dtype, device_map="auto"
)

# apply the AWQ results
apply_awq(model, awq_results)

model = quantize_opt_model(
    model,
    w_n_bits=q_config["w_n_bits"],
    a_n_bits=q_config["a_n_bits"],
    act_quant=q_config["act_quant"],
    group_size=q_config["q_group_size"],
)


In [53]:
torch.cuda.empty_cache()
model.cuda()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): QuantizedLinear()
          (fc2): QuantizedLinear()
          (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
 

In [54]:
# Evaluate the model
model_perplexity = evaluate(model, tokenizer)
model_size = get_model_size(model, data_width=q_config["w_n_bits"], group_size=q_config["q_group_size"])
print(f"\nmodel perplexity: {model_perplexity:.2f}")
print(f"model size: {model_size/MiB:.2f} MiB")

evaluating...: 100%|██████████| 40/40 [00:21<00:00,  1.86it/s]


model perplexity: 23815.02
model size: 248.04 MiB


In [42]:

print("hi")

hi


## RANDOM STUFF

In [43]:
from awq.quantize.pre_quant_lester import get_blocks

In [44]:
for layer in get_blocks(model):
    # layer.fc1
    # layer.fc2
    # print("hi")
    break

In [45]:
from copy import deepcopy

x = deepcopy(layer)

In [46]:
x.fc1.weight.data[:] = 1.0
# x.fc1.weight.data

layer.fc1.weight.data

tensor([[ 3.1622e-04,  2.4802e-05, -1.9221e-04,  ...,  1.8563e-04,
          6.4971e-04,  2.0110e-04],
        [ 5.2574e-05,  2.9792e-04,  3.6802e-04,  ..., -6.5032e-04,
         -3.0992e-04, -1.0161e-04],
        [-2.0619e-04, -1.8504e-04,  1.9033e-04,  ..., -2.7781e-04,
          5.7877e-06,  3.2990e-04],
        ...,
        [ 3.4242e-04,  1.7121e-04, -1.6377e-04,  ...,  2.4482e-04,
         -1.8939e-04,  2.0787e-04],
        [-5.0239e-05,  1.1722e-04, -1.1722e-04,  ...,  1.6503e-04,
          1.0045e-04, -2.8701e-04],
        [-4.2769e-05,  0.0000e+00,  2.5234e-04,  ..., -1.2389e-05,
         -2.8908e-05,  9.9112e-05]], device='cuda:0')

In [47]:
isinstance(layer, OPTDecoderLayer)

True